In [1]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [2]:
#imports

import numpy as np


In [ ]:
# Task 1, Preparing the dataset
# WOS-11967 is a list of 11967 abstracts from scientific papers alongside 2 labels for each instance - (main domain, sub domain)

"""
1. Load the raw text and labels. (each abstract is on each line)
2. Tokenize the abstracts.
3. Build a vocabulary for word-level LSTM. (LSTM needs words, not tokens like BERT)
4. Encode each abstract as a padded sequence of word IDs.
5. Track the true sequence lengths (before padding).
6. Split into train/val/test sets (categorized by main label).
7. Save processed tensors to disk for later use.

outputs:
- wos_lstm_train.pt
- wos_lstm_val.pt
- wos_lstm_test.pt
- wos_bert_train.pt
- wos_bert_val.pt
- wos_bert_test.pt
"""

# NOTE: THE CODE WAS RUN ON GOOGLE COLAB SEPARATELY.

import os
import re
import argparse
from collections import Counter

import numpy as np
import torch
from sklearn.model_selection import train_test_split

# load raw dataset files
"""
returns
- texts: list[abstract]
- labels_main: np.array(11967)
- labels_sub: np.array(11967)
"""

def load_wos_txt():
    x_path = "./dataset/X.txt"
    yl1_path = "./dataset/YL1.txt"
    yl2_path = "./dataset/YL2.txt"

    with open(x_path, encoding="utf-8") as f:
        texts = [line.strip() for line in f]

    with open(yl1_path, encoding="utf-8") as f:
        labels_main = [int(line.strip()) for line in f]

    with open(yl2_path, encoding="utf-8") as f:
        labels_sub = [int(line.strip()) for line in f]

    return texts, np.array(labels_main, dtype=np.int64), np.array(labels_sub, dtype=np.int64)


# tokenization, regex is to replace any non "regular" characters with spaces
_token_pattern = re.compile(r"[^a-z0-9]+")

def simple_tokenize(text):
    text = text.lower()
    text = _token_pattern.sub(" ", text)
    return text.split()

def tokenize_inputs(texts):
    return [simple_tokenize(t) for t in texts]

# super simple "embedding" method; go over all the words, and as long as they appear more than twice (prevent URLs and stuff like that),
# add to the vocab with id = len(vocab) (which means each new vocab word added, the length of vocab increases)
# keep <PAD> and <UNK> for padding (PAD in case abstract too short, UNK for "unknown" token in case an unknown word appears) 
def build_vocab(tokenized_texts, min_freq):
    counter = Counter()
    for toks in tokenized_texts:
        counter.update(toks)

    vocab = {"<PAD>": 0, "<UNK>": 1}
    for tok, freq in counter.items():
        if freq >= min_freq:
            vocab[tok] = len(vocab)
    return vocab

"""
output is
- input_ids: N x max_len matrix of our embeddings. Contains our custom embedding logic + PAD whenever the abstract length < max_len.
    obviously, if abstract length > max_len, it gets right truncated
- lengths: N length array of (actual) lengths of the abstracts. Of course, if ith abstract length > max_len, lengths[i] = max_len 
"""
def encode_and_pad(tokenized_texts, vocab, max_len=None):
    lengths_raw = np.array([len(toks) for toks in tokenized_texts], dtype=np.int64)

    if max_len is None:
        # the LSTM input is only as long as the 95% longest
        max_len = int(np.percentile(lengths_raw, 95))

    N = len(tokenized_texts)
    # here is the init of the "embeddings" matrix (initially full of <PAD> (0))
    input_ids = np.full((N, max_len), fill_value=vocab["<PAD>"], dtype=np.int64)
    lengths = np.zeros(N, dtype=np.int64)

    for i, toks in enumerate(tokenized_texts):
        # THE embedding step
        ids = [vocab.get(tok, vocab["<UNK>"]) for tok in toks]
        truncated = ids[:max_len]
        input_ids[i, :len(truncated)] = truncated
        lengths[i] = min(len(ids), max_len)

    return input_ids, lengths, max_len



# very simply use the bert tokenizer for bert. Assume same 95th percentile max length.
# input_ids, same concept as above, but when fed into BERT, BERT internally maps the inputs
def build_bert_features(texts, top95length):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    encodings = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=top95length,
        return_tensors="pt"
    )
    return encodings["input_ids"], encodings["attention_mask"]



# splits
def split_indices(labels_main, test_size=0.25, val_size=0.25, seed=67):
    """
    Returns train_idx, val_idx, test_idx (NumPy arrays of indices),
    stratified by main labels.
    """
    N = len(labels_main)
    indices = np.arange(N)

    train_idx, temp_idx = train_test_split(
        indices,
        test_size=test_size,
        stratify=labels_main,
        random_state=seed,
    )

    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=val_size,
        stratify=labels_main[temp_idx],
        random_state=seed,
    )

    return train_idx, val_idx, test_idx



def main():
    # 1. Load raw data
    texts, labels_main, labels_sub = load_wos_txt()
    print(f"Loaded {len(texts)} abstracts.")

    # 2. LSTM pipeline: tokenize -> vocab -> encode and pad
    tokenized = tokenize_inputs(texts)
    print("Example tokens:", tokenized[0][:5])

    vocab = build_vocab(tokenized, min_freq=3)
    print(f"LSTM vocab size (including PAD/UNK): {len(vocab)}")

    lstm_input_ids, lstm_lengths, max_len_lstm_used = encode_and_pad(
        tokenized,
        vocab
    )
    print(f"LSTM encoded shape: {lstm_input_ids.shape} (max_len={max_len_lstm_used})")

    # 3. BERT pipeline: tokenizer -> input_ids + attention mask
    bert_input_ids, bert_attention_mask = build_bert_features(
        texts,
        top95length=max_len_lstm_used
    )
    print(f"BERT encoded shape: {bert_input_ids.shape} (max_len={max_len_lstm_used})")

    # 4. Shared train/val/test indices
    train_idx, val_idx, test_idx = split_indices(
        labels_main
    )

    # 5. Build LSTM splits
    train_lstm = {
        "input_ids": torch.from_numpy(lstm_input_ids[train_idx]),
        "lengths": torch.from_numpy(lstm_lengths[train_idx]),
        "label_main": torch.from_numpy(labels_main[train_idx]),
        "label_sub": torch.from_numpy(labels_sub[train_idx]),
    }
    val_lstm = {
        "input_ids": torch.from_numpy(lstm_input_ids[val_idx]),
        "lengths": torch.from_numpy(lstm_lengths[val_idx]),
        "label_main": torch.from_numpy(labels_main[val_idx]),
        "label_sub": torch.from_numpy(labels_sub[val_idx]),
    }
    test_lstm = {
        "input_ids": torch.from_numpy(lstm_input_ids[test_idx]),
        "lengths": torch.from_numpy(lstm_lengths[test_idx]),
        "label_main": torch.from_numpy(labels_main[test_idx]),
        "label_sub": torch.from_numpy(labels_sub[test_idx]),
    }

    # 6. Build BERT splits
    train_bert = {
        "input_ids": bert_input_ids[train_idx],
        "attention_mask": bert_attention_mask[train_idx],
        "label_main": torch.from_numpy(labels_main[train_idx]),
        "label_sub": torch.from_numpy(labels_sub[train_idx]),
    }
    val_bert = {
        "input_ids": bert_input_ids[val_idx],
        "attention_mask": bert_attention_mask[val_idx],
        "label_main": torch.from_numpy(labels_main[val_idx]),
        "label_sub": torch.from_numpy(labels_sub[val_idx]),
    }
    test_bert = {
        "input_ids": bert_input_ids[test_idx],
        "attention_mask": bert_attention_mask[test_idx],
        "label_main": torch.from_numpy(labels_main[test_idx]),
        "label_sub": torch.from_numpy(labels_sub[test_idx]),
    }

    # 7. Save everything
    torch.save(train_lstm, "wos_lstm_train.pt")
    torch.save(val_lstm, "wos_lstm_val.pt")
    torch.save(test_lstm, "wos_lstm_test.pt")
    print("Saved LSTM splits: wos_lstm_train.pt, wos_lstm_val.pt, wos_lstm_test.pt")

    torch.save(train_bert, "wos_bert_train.pt")
    torch.save(val_bert, "wos_bert_val.pt")
    torch.save(test_bert, "wos_bert_test.pt")
    print("Saved BERT splits: wos_bert_train.pt, wos_bert_val.pt, wos_bert_test.pt")

    torch.save(
        {
            "vocab": vocab,
            "min_freq": 3,
            "max_len_lstm": max_len_lstm_used,
        },
        "wos_vocab.pt"
    )
    print("Saved vocab: wos_vocab.pt")


main()

ValueError: mount failed

In [3]:
#useful functions
def l2_loss(y, yh):
  return 0.5 * (yh - y)**2

def l2_loss_grad(y, yh):
  return yh - y

def cross_entropy(y, yh):
  return -np.sum(y * np.log(yh + 1e-12))

# note that this is true only for dL/dz, L = loss(softmax(z))
def cross_entropy_grad(y, yh):
  return yh - y

def relu(x):
  return np.maximum(0, x)
   
def relu_grad(x):
    return (x > 0).astype(float)

def leaky_relu(x, alpha=0.1):
   return np.maximum(alpha*x, x)

def leaky_relu_grad(x, alpha=0.1):
    grad = np.ones_like(x)
    grad[x < 0] = alpha
    return grad

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    s = sigmoid(x)
    return s * (1 - s)
  
def tanh(x):
  return np.tanh(x)

def tanh_grad(x):
  t = np.tanh(x)
  return 1 - t * t

def linear(x):
  return x

def linear_grad(x):
  return np.ones_like(x)

def softmax(x):
  z = x - np.max(x)
  e = np.exp(z)
  return e / np.sum(e)

In [8]:
#task 2 implement LSTM

class Cell:
    def __init__(self, input_size, hidden_size, b_f=None, b_i=None, b_o=None, b_c=None):
        #random weight matrices for now
        self.W_f = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.W_i = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.W_c = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.W_o = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        #initialize biases
        self.b_f = b_f if b_f is not None else np.ones(hidden_size)  
        self.b_i = b_i if b_i is not None else np.zeros(hidden_size)
        self.b_o = b_o if b_o is not None else np.zeros(hidden_size)
        self.b_c = b_c if b_c is not None else np.zeros(hidden_size)

    #forget gate
    def get_f_t(self, h_prev, x): 
        """"
        computes the forget value given h_{t-1} (prev hidden state), x_t (input), b_f (current forget bias)
    
        """
        concat = np.concatenate([h_prev, x], axis=-1)
        return sigmoid(np.dot(self.W_f, concat) + self.b_f)
    
    #input gate
    def get_i_t(self, h_prev, x):
        concat = np.concatenate([h_prev, x], axis=-1)
        return sigmoid(np.dot(self.W_i, concat) + self.b_i)
    
    def get_c_hat(self, h_prev, x):
        concat = np.concatenate([h_prev, x], axis=-1)
        return tanh(np.dot(self.W_c, concat) + self.b_c)
    
    def get_c_t(self, c_prev, h_prev, x):
        c_hat = self.get_c_hat(h_prev, x)
        i_t = self.get_i_t(h_prev, x)
        forget = self.get_f_t(h_prev, x)
        return forget * c_prev + i_t * c_hat
    
    #output gate
    def get_o_t(self, h_prev, x):
        concat = np.concatenate([h_prev, x], axis=-1)
        return sigmoid(np.dot(self.W_o, concat) + self.b_o)

    def get_h(self, o_t, c_t):
        return o_t * tanh(c_t)
    
    def forward(self, h_prev, c_prev, x): # <<<<<<<<<<<<<<<---------------------------- use this!
        """ 
        computes a forward pass of the entire cell
        returns: h_t, c_t
        """
        f_t = self.get_f_t(h_prev, x)
        i_t = self.get_i_t(h_prev, x)
        c_t = self.get_c_t(c_prev, h_prev, x)
        o_t = self.get_o_t(h_prev, x)
        h_t = self.get_h(o_t, c_t)
        self.cache = {
            'h_prev': h_prev, 'c_prev': c_prev, 'x': x,
            'f_t': f_t, 'i_t': i_t, 'c_hat': c_hat,
            'c_t': c_t, 'o_t': o_t, 'concat': concat
        }
        return h_t, c_t
    
    def backward(self, dh_next, dc_next):
        """
        dh_next: gradient of loss w.r.t h_t (from output or next timestep)
        dc_next: gradient of loss w.r.t C_t (from next timestep)
        returns: dx, dh_prev, dc_prev
        and stores dW/db for optimizer
        """
        # retrieve cached values
        f_t = self.cache['f_t']
        i_t = self.cache['i_t']
        c_hat = self.cache['c_hat']
        o_t = self.cache['o_t']
        c_prev = self.cache['c_prev']
        concat = self.cache['concat']

        c_t = self.cache['c_t']

        # derivative of loss w.r.t c_t (total)
        dc_t = dh_next * o_t * (1 - np.tanh(c_t)**2) + dc_next

        # derivatives w.r.t gates before activation
        do = dh_next * np.tanh(c_t)
        di = dc_t * c_hat
        df = dc_t * c_prev
        dc_hat = dc_t * i_t

        # apply activation derivatives
        dZ_o = do * o_t * (1 - o_t)
        dZ_i = di * i_t * (1 - i_t)
        dZ_f = df * f_t * (1 - f_t)
        dZ_c = dc_hat * (1 - c_hat**2)

        # gradients w.r.t weights and biases
        self.dW_o = np.outer(dZ_o, concat)
        self.dW_i = np.outer(dZ_i, concat)
        self.dW_f = np.outer(dZ_f, concat)
        self.dW_c = np.outer(dZ_c, concat)

        self.db_o = dZ_o
        self.db_i = dZ_i
        self.db_f = dZ_f
        self.db_c = dZ_c

        # gradient w.r.t concat
        dconcat = (self.W_o.T @ dZ_o +
                   self.W_i.T @ dZ_i +
                   self.W_f.T @ dZ_f +
                   self.W_c.T @ dZ_c)

        # split into dx and dh_prev
        hidden_size = self.cache['h_prev'].shape[0]
        dx = dconcat[hidden_size:]
        dh_prev = dconcat[:hidden_size]
        dc_prev = dc_t * f_t

        return dx, dh_prev, dc_prev


class LSTM: # note: I hardcoded seq2vec based on the assignment description

    def __init__(self, input_size, hidden_size): #initialize a network with #cell_count cells
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.cell = Cell(input_size, hidden_size)
        self.h_cache = []   # stores all h_t
        self.c_cache = []   # stores all c_t
        self.gates_cache = []  # optional, stores f_t, i_t, o_t, c_hat

    def fit(self, X_seqs, Y_vecs, batch_size=1, lr=0.01, epochs=10):
        """
        X_seqs: list of sequences, each sequence is array of shape (seq_len, input_size)
        Y_vecs: list of target vectors, each of shape (output_size,)
        batch_size: number of sequences per batch (can start with 1)
        lr: learning rate
        epochs: number of training passes
        """
        for epoch in range(epochs):
            total_loss = 0

            # optionally shuffle data
            for i in range(0, len(X_seqs), batch_size):
                batch_X = X_seqs[i:i+batch_size]
                batch_Y = Y_vecs[i:i+batch_size]

                # initialize gradient accumulators
                self.cell.reset_gradients()  #

                for x_seq, y_vec in zip(batch_X, batch_Y):
                    # 1️ Forward pass through the sequence
                    h_prev = np.zeros(self.hidden_size)
                    c_prev = np.zeros(self.hidden_size)

                    for x_t in x_seq:
                        h_prev, c_prev = self.cell.forward(h_prev, c_prev, x_t)

                    # h_prev is now the final hidden state -> seq2vec
                    y_pred = h_prev  # or pass through a linear layer if you want

                    # 2️ Compute loss (MSE example)
                    loss = 0.5 * np.sum((y_pred - y_vec)**2)
                    total_loss += loss

                    # 3️ Backward pass (BPTT)
                    dh = y_pred - y_vec
                    dc = np.zeros_like(c_prev)

                    for t in reversed(range(len(x_seq))):
                        # call cell.backward() with dh and dc
                        dx, dh, dc = self.cell.backward(dh, dc)
                        # cell.backward accumulates gradients internally

                # 4️ Update weights after batch
                self.cell.W_f -= lr * self.cell.dW_f
                self.cell.b_f -= lr * self.cell.db_f
                self.cell.W_i -= lr * self.cell.dW_i
                self.cell.b_i -= lr * self.cell.db_i
                self.cell.W_c -= lr * self.cell.dW_c
                self.cell.b_c -= lr * self.cell.db_c
                self.cell.W_o -= lr * self.cell.dW_o
                self.cell.b_o -= lr * self.cell.db_o

            print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

    def predict(self, X_seq):
        h_prev = np.zeros(self.hidden_size)
        c_prev = np.zeros(self.hidden_size)

        for x_t in X_seq:
            h_prev, c_prev = self.cell.forward(h_prev, c_prev, x_t)

        y_pred = h_prev
        return y_pred
    
    def evaluate_acc(self, X_seq, Y_vec):
        y_pred = self.predict(X_seq)
        correct=0
        for i in range(len(y_pred)):
            if y_pred[i] == Y_vec[i]:
                correct+=1
        return correct/len(Y_vec)


#TODO implement testing
    

In [2]:
# TASK 2 part b: BERT model
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

# Example dataset (replace with your WOS data)
texts = [
    "This is an example abstract 1.",
    "Another abstract for testing.",
    "Yet another example abstract."
]
labels = [0, 2, 1]  # integer class labels for each abstract

# Parameters
num_labels = 7          # number of top-level classes in WOS
max_len = 128           # max token length
batch_size = 2
epochs = 3
lr = 2e-5

# Load pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# Tokenize all texts
encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
labels_tensor = torch.tensor(labels)

# Create dataset and dataloader
dataset = TensorDataset(input_ids, attention_mask, labels_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)

# Training loop
model.train()
for epoch in range(epochs):
    for batch in dataloader:
        b_input_ids, b_attention_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} completed.")

# Evaluation / Prediction
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)

print("Predicted classes:", preds)
print("True labels:", labels_tensor)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3 completed.
Epoch 2/3 completed.
Epoch 3/3 completed.
Predicted classes: tensor([4, 1, 1])
True labels: tensor([0, 2, 1])
